In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf 
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
df = pd.read_json('sarcasm.json', lines= True)

In [3]:
df.shape

(28619, 3)

In [4]:
df['headline'][3]

'inclement weather prevents liar from getting to work'

In [5]:
df

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...
...,...,...,...
28614,1,jews to celebrate rosh hashasha or something,https://www.theonion.com/jews-to-celebrate-ros...
28615,1,internal affairs investigator disappointed con...,https://local.theonion.com/internal-affairs-in...
28616,0,the most beautiful acceptance speech this week...,https://www.huffingtonpost.com/entry/andrew-ah...
28617,1,mars probe destroyed by orbiting spielberg-gat...,https://www.theonion.com/mars-probe-destroyed-...


In [6]:
sent = ['I am learning Python', 'I am learning Deep Learning', 'I love dogs', 'I love cats' ]

In [7]:
sent[1]

'I am learning Deep Learning'

In [8]:
type(sent)

list

In [9]:
sent

['I am learning Python',
 'I am learning Deep Learning',
 'I love dogs',
 'I love cats']

In [10]:
token  = Tokenizer(10, )

In [11]:
token.fit_on_texts(sent)

In [12]:
token.word_index

{'i': 1,
 'learning': 2,
 'am': 3,
 'love': 4,
 'python': 5,
 'deep': 6,
 'dogs': 7,
 'cats': 8}

In [13]:
sent_seq = token.texts_to_sequences(sent)

In [14]:
print(sent_seq)

[[1, 3, 2, 5], [1, 3, 2, 6, 2], [1, 4, 7], [1, 4, 8]]


In [15]:
sent = ['I am learning Python', 'I am learning Deep Learning', 'I love dogs', 'I love cats' ]

In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
sent_final = pad_sequences( sent_seq, truncating= 'post', padding= 'post', maxlen= 10 )

In [18]:
sent_final

array([[1, 3, 2, 5, 0, 0, 0, 0, 0, 0],
       [1, 3, 2, 6, 2, 0, 0, 0, 0, 0],
       [1, 4, 7, 0, 0, 0, 0, 0, 0, 0],
       [1, 4, 8, 0, 0, 0, 0, 0, 0, 0]])

## Project : Sarcasm Detection

In [19]:
# 1. Convert headlines into a list.
# 2. Convert all labels into a list


In [20]:
# 
sentences = []
labels    = []

df.shape

(28619, 3)

In [21]:
sentences = df['headline'].to_list()

In [22]:
labels = df['is_sarcastic'].to_list()

In [23]:
# Create Training and Testing Datasets   => 90 : 10 

In [24]:
train = int(df.shape[0] * 90 / 100)
train

25757

In [25]:
# Training Dataset
sentences_train = sentences[ : train]
labels_train    = labels[ : train]

In [26]:
# Testing Dataset

sentences_test = sentences[train : ]
labels_test   = labels[train : ]

In [27]:
# Create Word Index

In [28]:
token = Tokenizer(num_words= 10000, oov_token= 'UNK')

In [29]:
token.fit_on_texts(sentences_train)

In [30]:
# token.word_index

In [31]:
train_seq = pad_sequences(token.texts_to_sequences(sentences_train),
                          maxlen= 50, 
                          padding= 'post', 
                          truncating= 'post')

In [32]:
train_seq.shape

(25757, 50)

In [33]:
test_seq = pad_sequences(token.texts_to_sequences(sentences_test),
                          maxlen= 50, 
                          padding= 'post', 
                          truncating= 'post')

In [34]:
test_seq.shape

(2862, 50)

In [35]:
# Convert labels also in array
train_label = np.array(labels_train)
test_label = np.array(labels_test)

In [36]:
test_label

array([0, 1, 0, ..., 0, 1, 1])

In [37]:
# Build a Model

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, GlobalAvgPool1D, Flatten


In [39]:
model = Sequential()

In [40]:
# Input Layer
model.add(Embedding(10000, input_length= 50, output_dim = 16))
model.add(GlobalAvgPool1D())

# First Hidden Layer
model.add(Dense(128, activation= 'relu'))
model.add(Dropout(0.25))

# Second Hidden Layer
model.add(Dense(64, activation= 'relu'))
model.add(Dropout(0.25))

# Output Layer
model.add(Dense(1, activation= 'sigmoid'))

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 16)            160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 128)               2176      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                        

In [42]:
# Compile model
model.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics = ['accuracy'])

In [43]:
# Train the Model
model.fit(train_seq, train_label, epochs= 10, validation_data= (test_seq, test_label))

Epoch 1/10


805/805 [==============================] - 7s 6ms/step - loss: 0.4757 - accuracy: 0.7472 - val_loss: 0.3549 - val_accuracy: 0.8421
Epoch 2/10
805/805 [==============================] - 4s 5ms/step - loss: 0.2740 - accuracy: 0.8904 - val_loss: 0.3454 - val_accuracy: 0.8487
Epoch 3/10
805/805 [==============================] - 4s 5ms/step - loss: 0.2169 - accuracy: 0.9179 - val_loss: 0.3878 - val_accuracy: 0.8375
Epoch 4/10
805/805 [==============================] - 4s 5ms/step - loss: 0.1851 - accuracy: 0.9318 - val_loss: 0.3709 - val_accuracy: 0.8491
Epoch 5/10
805/805 [==============================] - 4s 5ms/step - loss: 0.1603 - accuracy: 0.9429 - val_loss: 0.3822 - val_accuracy: 0.8438
Epoch 6/10
805/805 [==============================] - 4s 5ms/step - loss: 0.1426 - accuracy: 0.9501 - val_loss: 0.4180 - val_accuracy: 0.8414
Epoch 7/10
805/805 [==============================] - 4s 5ms/step - loss: 0.1268 - accuracy: 0.9562 - val_loss: 0.4464 - val_accuracy: 0.8389
Epoc

In [44]:
test = ["Where are women judges in high courts ?"]

test = pad_sequences(token.texts_to_sequences(test),
                          maxlen= 50, 
                          padding= 'post', 
                          truncating= 'post')


In [45]:
test1 = ["soooo intelligent!!!"]

test1 = pad_sequences(token.texts_to_sequences(test1),
                          maxlen= 50, 
                          padding= 'post', 
                          truncating= 'post')

In [46]:
model.predict(test).round()

1/1 [==============================] - 0s 117ms/step


array([[0.]], dtype=float32)

In [47]:
model.predict(test1).round()

1/1 [==============================] - 0s 30ms/step


array([[1.]], dtype=float32)